### Case Study A

In [38]:
# Importing Python libraries for data manipulation
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [39]:
# Reading in the data
provisioning_data = pd.read_excel('provisioning-data.xlsx')

In [40]:
provisioning_data.head()

,Loan_Id,Date_Of_Activity,Credits_End_Date,Outstanding_Balance,customerStatus,parProductType
0,1239226,2019-06-30,2018-10-07,0.0,Finished Payment,SHS
1,1250598,2019-06-30,2019-02-20,0.0,Finished Payment,SHS
2,1250602,2019-06-30,2018-08-16,0.0,Finished Payment,SHS
3,1251521,2019-06-30,2019-01-14,0.0,Finished Payment,SHS
4,1251524,2019-06-30,2017-12-21,700.0,Blocked,SHS


### Understanding the size of the data

In [41]:
provisioning_data.shape

(46237, 6)

### Creating a new column for: Days Without Light

In [42]:
provisioning_data['Days Without Light'] = provisioning_data['Date_Of_Activity'] - provisioning_data['Credits_End_Date']

In [43]:
# Convert the values of "Days Without Light" column to integer values
provisioning_data['Days Without Light'] = provisioning_data['Days Without Light'].astype('timedelta64[D]').astype(int)

In [44]:
# Check the first 5 rows of the data
provisioning_data.head()

,Loan_Id,Date_Of_Activity,Credits_End_Date,Outstanding_Balance,customerStatus,parProductType,Days Without Light
0,1239226,2019-06-30,2018-10-07,0.0,Finished Payment,SHS,266
1,1250598,2019-06-30,2019-02-20,0.0,Finished Payment,SHS,130
2,1250602,2019-06-30,2018-08-16,0.0,Finished Payment,SHS,318
3,1251521,2019-06-30,2019-01-14,0.0,Finished Payment,SHS,167
4,1251524,2019-06-30,2017-12-21,700.0,Blocked,SHS,556


### A function to group customers by: No of days without Light

In [45]:
# The value returned by this function is used to represent the various categories of customers based on their Days without light
def customers_group(days):
    if (days >= 0) & (days <= 31):
        value = '0-31'
    elif (days > 31) & (days <= 62):
        value = '32-62'
    elif (days > 62) & (days <= 93):
        value =  '63-93'
    elif (days > 93) & (days <= 124):
        value = '94-124'
    elif (days > 124) & (days <= 155):
        value = '125-155'
    else:
        value = '156+'
    return value

In [46]:
# Applying the above function on the column: Days Without Light
provisioning_data['Days Without Light'] = provisioning_data['Days Without Light'].apply(customers_group)

In [47]:
provisioning_data.head()

,Loan_Id,Date_Of_Activity,Credits_End_Date,Outstanding_Balance,customerStatus,parProductType,Days Without Light
0,1239226,2019-06-30,2018-10-07,0.0,Finished Payment,SHS,156+
1,1250598,2019-06-30,2019-02-20,0.0,Finished Payment,SHS,125-155
2,1250602,2019-06-30,2018-08-16,0.0,Finished Payment,SHS,156+
3,1251521,2019-06-30,2019-01-14,0.0,Finished Payment,SHS,156+
4,1251524,2019-06-30,2017-12-21,700.0,Blocked,SHS,156+


### Funtion to computer the Provisioning Percentage

In [48]:
# This function computes and return the provisioning percentage for the different categories of of customers based on their
# number of days without light
def provisioning_percentage():
    for value in provisioning_data['Days Without Light']:
        if value == '0-31':
            return 0/100
        elif value == '32-62':
            return 25/100
        elif value == '63-93':
            return 50/100
        elif value == '94-124':
            return 65/100
        elif value == '125-155':
            return 85/100
        elif value == '156+':
            return 1
        else:
            break
    for value in provisioning_data['customerStatus']:
        if value == 'Cancelled':
            return 1
        else:
            break

### Adding a column for the "Provisioning Amount"

In [49]:
# The function above - provisioning_percentage() is called in the line of code below to get the percentage that will be used
# to to compute the provisioning amount
provisioning_data['Provisioning Amount'] = provisioning_percentage() * provisioning_data['Outstanding_Balance']

In [50]:
provisioning_data.head()

,Loan_Id,Date_Of_Activity,Credits_End_Date,Outstanding_Balance,customerStatus,parProductType,Days Without Light,Provisioning Amount
0,1239226,2019-06-30,2018-10-07,0.0,Finished Payment,SHS,156+,0.0
1,1250598,2019-06-30,2019-02-20,0.0,Finished Payment,SHS,125-155,0.0
2,1250602,2019-06-30,2018-08-16,0.0,Finished Payment,SHS,156+,0.0
3,1251521,2019-06-30,2019-01-14,0.0,Finished Payment,SHS,156+,0.0
4,1251524,2019-06-30,2017-12-21,700.0,Blocked,SHS,156+,700.0


### Computing the Opening Balance
##### The opening balance is the cumulative sum of the  Provisioning Amount  across all the buckets at the end of the month.

In [51]:
# Extracting the data for 30th of June (End of the month) as provided in the data
cumulative_sum_data1 = provisioning_data[provisioning_data['Date_Of_Activity'] == '2019-06-30']

In [52]:
# Aggregating the cumulative sum of the provisioning amount for the category of data above using the Groupby function of pandas
cumulative_sum_data1 = cumulative_sum_data1.groupby(['Days Without Light', 'parProductType'])['Provisioning Amount'].sum()

In [53]:
# The opening balance can be seen in the result below. The name for the column will be fixed in the next lines of code
cumulative_sum_data1

Days Without Light  parProductType
0-31                 SHS              1.348298e+06
                     TV               3.708519e+06
125-155              SHS              3.961603e+05
                     TV               3.853005e+05
156+                 SHS              5.552157e+06
                     TV               1.748004e+07
32-62                SHS              5.308608e+05
                     TV               9.623837e+05
63-93                SHS              4.681399e+05
                     TV               6.139079e+05
94-124               SHS              5.067621e+05
                     TV               4.578344e+05
Name: Provisioning Amount, dtype: float64

### Computing the Closing Balance
##### The closing balance is the  Cumulative sum of the Provisioning Amount across all the buckets at the time of reporting, 
##### in our case this would be 22nd of July from the data Provided 

In [54]:
# Extracting the data for 22nd of July (the time of reporting) as provided in the data
cumulative_sum_data2 = provisioning_data[provisioning_data['Date_Of_Activity'] == '2019-07-22']

In [55]:
# Aggregating the cumulative sum of the provisioning amount for the category of the data above using the Groupby function of pandas
cumulative_sum_data2 = cumulative_sum_data2.groupby(['Days Without Light', 'parProductType'])['Provisioning Amount'].sum()

In [56]:
cumulative_sum_data2

Days Without Light  parProductType
0-31                 SHS              1.142216e+06
                     TV               4.387236e+06
125-155              SHS              4.847306e+05
                     TV               4.016043e+05
156+                 SHS              5.530449e+06
                     TV               1.768103e+07
32-62                SHS              5.500859e+05
                     TV               9.705940e+05
63-93                SHS              4.284915e+05
                     TV               6.767160e+05
94-124               SHS              4.359904e+05
                     TV               5.381987e+05
Name: Provisioning Amount, dtype: float64

### Converting the cumulative sum data above into a Pandas Dataframe
##### Both  data are pandas series objects. Converint into a Dataframe will help us combine both data into one comprehensive data file.

In [57]:
data1 = pd.DataFrame(cumulative_sum_data1)
data2 = pd.DataFrame(cumulative_sum_data2)

In [58]:
# Rename "Provisioning Amount" on both data to "Opening Provisioning Amount" & "Closing Provisioning Amount" as 
# presented in the final output file  of this project

In [59]:
data1.head()

Provisioning Amount
Days Without Light parProductType                     
0-31                SHS                   1.348298e+06
                    TV                    3.708519e+06
125-155             SHS                   3.961603e+05
                    TV                    3.853005e+05
156+                SHS                   5.552157e+06

In [60]:
data1 = data1.rename(columns = {'Provisioning Amount': 'Opening Provisioning Amount'})
data2 = data2.rename(columns = {'Provisioning Amount': 'Closing Provisioning Amount'})

### Merge Both data above into a comprehensive data file.
##### The merge operation was done on two keys needed to present our final output

In [61]:
output_data = pd.merge(data1, data2, on = ['parProductType','Days Without Light'])

In [62]:
output_data.head()

,,Opening Provisioning Amount,Closing Provisioning Amount
parProductType,Days Without Light,,
SHS,0-31,1.348298e+06,1.142216e+06
TV,0-31,3.708519e+06,4.387236e+06
SHS,125-155,3.961603e+05,4.847306e+05
TV,125-155,3.853005e+05,4.016043e+05
SHS,156+,5.552157e+06,5.530449e+06


### Compute cost of provisioning

In [63]:
cost_of_provisioning = output_data['Closing Provisioning Amount'] - output_data['Opening Provisioning Amount']

In [64]:
cost_of_provisioning

parProductType  Days Without Light
 SHS            0-31                 -206081.670000
 TV             0-31                  678717.410476
 SHS            125-155                88570.280000
 TV             125-155                16303.761905
 SHS            156+                  -21707.945714
 TV             156+                  200991.507143
 SHS            32-62                  19225.050000
 TV             32-62                   8210.251905
 SHS            63-93                 -39648.430000
 TV             63-93                  62808.134286
 SHS            94-124                -70771.660000
 TV             94-124                 80364.294286
dtype: float64

### Summary:

#### It can be deduced from the cost of provisioning above that customers using Solar Panels with just ligt bulb (SHS), tend to incure more loss  for PEG Africa as the number of days they remained without light increases across the different buckets.

### CASE STUDY B

#### Problem 1:

<p>
1)The customers statuses are being overwritten once a customer finishes paying and hence one isn’t able to monitor the customer 
movement behavior in the period of their payment. If a customer was blocked and then becomes active the new status will overwrite
the blocked status and therefore if you wanted to study how many times this customer was blocked or has been active you don’t 
have this information. How do we write code in Python that is able to preserve the statuses and not overwrite them Once a 
customer finishes to pay? 
</p>

### A function to preserve the statuses of the customers.
<p> This function is called once a customer finishes his/her payment. The output of the function ensures the status of the
customer is preserved before and after the circle of payment.</p>

#### Problem 2:

In [65]:
# Read in the customer token information data
customer_token_info = pd.read_csv('Customer_Token_Information.csv')

In [66]:
customer_token_info.head()

,contractId,AccountNumber,CreatedAt,Amount,Count,Duration
0,1307022,142325110,9/1/2017 15:54,100,0,5
1,1307374,92511041,9/2/2017 16:31,100,0,5
2,1307721,202399520,9/4/2017 11:08,100,0,5
3,1307764,676152506,9/4/2017 12:12,100,0,5
4,1307862,698858327,9/4/2017 15:00,100,0,5


### A function to keep track of A customer's Day of Activity

<p> This function keeps track of the Date of Activity of each of the customer. The output helps to understand each customer’s behavior throughout their credit life cycle until they finish the loan</p>
<p> This function is only called when the Date of Activity is need. This means there will be no need to log the Date of Activity of each customers daily which increases the cost of data storage as well as results in performance issues.</p>

In [67]:
def date_activity_tracker():
    # The date of activity can be computed by sustracting the Token Dutation from the Credit_End_Date
    date_of_activity = provisioning_data['Credits_End_Date'] - customer_token_info['Duration'].astype('timedelta64[D]')
    date_of_activity = pd.DataFrame(data = date_of_activity, columns = ['Date_Of_Activity'])
    return date_of_activity

### Output of the function when called

In [68]:
date_activity_tracker().head()

,Date_Of_Activity
0,2018-10-02
1,2019-02-15
2,2018-08-11
3,2019-01-09
4,2017-12-16


### Delete the original Date of Activity

<p> Now that we have a function that keeps track and store the Date of Activity for each customers
and outputs that when needed, we can now delete the Date_of_Activity column from the provisioning data.</p>

In [71]:
provisioning_data.drop(['Date_Of_Activity'], axis = 1, inplace = True)

In [72]:
provisioning_data.head()

,Loan_Id,Credits_End_Date,Outstanding_Balance,customerStatus,parProductType,Days Without Light,Provisioning Amount
0,1239226,2018-10-07,0.0,Finished Payment,SHS,156+,0.0
1,1250598,2019-02-20,0.0,Finished Payment,SHS,125-155,0.0
2,1250602,2018-08-16,0.0,Finished Payment,SHS,156+,0.0
3,1251521,2019-01-14,0.0,Finished Payment,SHS,156+,0.0
4,1251524,2017-12-21,700.0,Blocked,SHS,156+,700.0


## Case Study C - SQL Assesment